In [7]:
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np

In [8]:
data = pd.read_csv('./in/copper_returns_5d_final.csv')
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

In [9]:
def fit_evaluate_sarima(series, order, seasonal_order):
    # Dividir en conjuntos de entrenamiento y prueba
    train, test = train_test_split(series, test_size=0.15, shuffle=False)

    # Ajustar el modelo SARIMA
    model = SARIMAX(train, order=order, seasonal_order=seasonal_order, enforce_stationarity=False, enforce_invertibility=False)
    fitted_model = model.fit(disp=False)

    # Predicciones
    predictions = fitted_model.get_forecast(steps=len(test))
    predicted_means = predictions.predicted_mean

    # Calcular el RMSE
    rmse = np.sqrt(mean_squared_error(test, predicted_means))

    return fitted_model.summary(), rmse

In [10]:
summary, rmse = fit_evaluate_sarima(data['LMCADY_acu_5d'], (1, 0, 1), (0, 1, 1, 12))

c:\Users\gapuj\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\gapuj\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\gapuj\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\gapuj\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monoto

In [11]:
print(summary)
print("RMSE: {:.4}".format(rmse))

                                     SARIMAX Results                                      
Dep. Variable:                      LMCADY_acu_5d   No. Observations:                 4708
Model:             SARIMAX(1, 0, 1)x(0, 1, 1, 12)   Log Likelihood               11140.496
Date:                            Fri, 19 Apr 2024   AIC                         -22272.992
Time:                                    20:49:16   BIC                         -22247.186
Sample:                                         0   HQIC                        -22263.917
                                           - 4708                                         
Covariance Type:                              opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.7655      0.008     95.012      0.000       0.750       0.781
ma.L1          0.0343      0.012   